# 食品制造优化 I

## 目标和前提条件

如果你渴望一个数学优化的挑战，可以尝试这个食品制造问题。你将学习如何为一种需要多种原料的产品创建最优的多期生产计划 - 每种原料都有不同的成本、限制和特性。

关于这种类型模型的更多信息可以在 H. P. Williams 的《数学规划中的模型构建》第五版第 253-254 页和第 296-298 页的示例 #1 中找到。

这个建模示例属于中级水平，我们假设你了解 Python 并熟悉 Gurobi Python API。此外，你还应该对构建数学优化模型有一定的了解。

**下载代码库** <br />
你可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一个制造商需要精炼几种原油并将它们混合在一起生产可以销售的食品。所需的原油可以分为两类：


| 类别 | 油类 |
| ------------- |-------------| 
| 植物油：| VEG 1<br>VEG 2 | 
| 非植物油：| OIL 1<br>OIL 2<br>OIL 3 |

制造商可以选择购买当月的原油和/或在期货市场上购买下月交付的原油。即期和期货市场的价格（美元/吨）如下：

| 月份 | VEG 1 | VEG 2 | OIL 1 | OIL 2 | OIL 3 |
| ------------- |-------------| -------------| -------------| -------------| -------------| 
| 一月 | 110 | 120 | 130 | 110 | 115 |
| 二月 | 130 | 130 | 110 | 90 | 115 |
| 三月 | 110 | 140 | 130 | 100 | 95 |
| 四月 | 120 | 110 | 120 | 120 | 125 |
| 五月 | 100 | 120 | 150 | 110 | 105 |
| 六月 | 90 | 100 | 140 | 80 | 135 |

还必须考虑一些额外的因素。这些包括：

1. 最终食品产品售价为 $\$150$ 每吨。
2. 每类油（植物油和非植物油）需要在不同的生产线上精炼。
3. 精炼能力有限，任何给定月份最多可以精炼 200 吨植物油和 250 吨非植物油。
4. 此外，精炼过程中没有废料，所以精炼的原油总量将等于可用的精炼油量。
5. 可以忽略油类精炼的成本。

除了上述精炼限制外，还有存储用于未来使用的原油数量的限制，每吨油的存储都有相关成本。限制是每种原油 1,000 吨，存储成本为每吨每月 $\$5$。制造商不能存储生产的食品产品或精炼油。

最终食品产品在给定硬度等级上的硬度必须在 3 到 6 之间。就模型而言，硬度呈线性混合，每种原油的硬度为：

| 油类 | 硬度 |
| ------------- |-------------| 
| VEG 1 | 8.8 |
| VEG 2 | 6.1 |
| OIL 1 | 2.0 |
| OIL 2 | 4.2 |
| OIL 3 | 5.0 |

在一月初，每种原油的库存为 500 吨。就模型而言，这也应该是六月末原油的库存水平。

根据上述信息，应该做出哪些月度采购和制造决策才能实现利润最大化？

---
## 模型构建

### 集合和索引

$t \in \text{Months}=\{\text{Jan},\text{Feb},\text{Mar},\text{Apr},\text{May},\text{Jun}\}$：月份集合。

$V=\{\text{VEG1},\text{VEG2}\}$：植物油集合。

$N=\{\text{OIL1},\text{OIL2},\text{OIL3}\}$：非植物油集合。

$o \in \text{Oils} = V \cup N$：油类集合。

### 参数

$\text{price} \in \mathbb{R}^+$：最终产品的销售价格。

$\text{init_store} \in \mathbb{R}^+$：初始库存量（吨）。

$\text{target_store} \in \mathbb{R}^+$：目标库存量（吨）。

$\text{holding_cost} \in \mathbb{R}^+$：每吨油每月的库存成本（美元/吨/月）。

$\text{veg_cap} \in \mathbb{R}^+$：植物油精炼装置容量（吨）。

$\text{oil_cap} \in \mathbb{R}^+$：非植物油精炼装置容量（吨）。

$\text{min_hardness} \in \mathbb{R}^+$：最终产品允许的最低硬度。

$\text{max_hardness} \in \mathbb{R}^+$：最终产品允许的最高硬度。

$\text{hardness}_o \in \mathbb{R}^+$：油类 $o$ 的硬度。

$\text{cost}_{t,o} \in \mathbb{R}^+$：月份 $t$ 时油类 $o$ 的估计购买价格。


### 决策变量

$\text{produce}_t \in \mathbb{R}^+$：月份 $t$ 生产的食品吨数。

$\text{buy}_{t,o} \in \mathbb{R}^+$：月份 $t$ 购买的油类 $o$ 吨数。

$\text{consume}_{t,o} \in \mathbb{R}^+$：月份 $t$ 使用的油类 $o$ 吨数。

$\text{store}_{t,o} \in \mathbb{R}^+$：月份 $t$ 储存的油类 $o$ 吨数。


### 目标函数

- **利润**：最大化规划周期的总利润（美元）。

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Months}}\text{price}*\text{produce}_t - \sum_{t \in \text{Months}}\sum_{o \in \text{Oils}}(\text{cost}_{t,o}*\text{consume}_{t,o} + \text{holding_cost}*\text{store}_{t,o})
\tag{0}
\end{equation}

### 约束条件

- **初始平衡**：一月购买的油类 $o$ 吨数和之前储存的油类数量应等于该月消耗和储存的吨数。

\begin{equation}
\text{init_store} + \text{buy}_{Jan,o} = \text{consume}_{Jan,o} + \text{store}_{Jan,o} \quad \forall o \in \text{Oils}
\tag{1}
\end{equation}

- **平衡**：月份 $t$ 购买的油类 $o$ 吨数和之前储存的油类数量应等于该月消耗和储存的吨数。

\begin{equation}
\text{store}_{t-1,o} + \text{buy}_{t,o} = \text{consume}_{t,o} + \text{store}_{t,o} \quad \forall (t,o) \in \text{Months} \setminus \{\text{Jan}\} \times \text{Oils}
\tag{2}
\end{equation}

- **库存目标**：规划周期末油类 $o$ 的库存量应达到目标。

\begin{equation}
\text{store}_{Jun,o} = \text{target_store} \quad \forall o \in \text{Oils}
\tag{3}
\end{equation}

- **精炼能力**：月份 $t$ 消耗的油类 $o$ 总吨数不能超过精炼能力。

\begin{equation}
\sum_{o \in V}\text{consume}_{t,o} \leq \text{veg_cap} \quad \forall t \in \text{Months}
\tag{4.1}
\end{equation}

\begin{equation}
\sum_{o \in N}\text{consume}_{t,o} \leq \text{oil_cap} \quad \forall t \in \text{Months}
\tag{4.2}
\end{equation}

- **硬度**：月份 $t$ 生产的食品硬度值应在容差范围内。

\begin{equation}
\text{min_hardness}*\text{produce}_t \leq \sum_{o \in \text{Oils}} \text{hardness}_o*\text{consume}_{t,o} \leq \text{max_hardness}*\text{produce}_t \quad \forall t \in \text{Months}
\tag{5}
\end{equation}

- **质量守恒**：月份 $t$ 消耗的油类总吨数应等于该月生产的食品吨数。

\begin{equation}
\sum_{o \in \text{Oils}}\text{consume}_{t,o} = \text{produce}_t \quad \forall t \in \text{Months}
\tag{6}
\end{equation}

---
## Python 实现

导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9

## 输入数据
我们定义模型的所有输入数据。

In [2]:
# 参数设置

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

cost = {
    ('Jan', 'VEG1'): 110,
    ('Jan', 'VEG2'): 120,
    ('Jan', 'OIL1'): 130,
    ('Jan', 'OIL2'): 110,
    ('Jan', 'OIL3'): 115,
    ('Feb', 'VEG1'): 130,
    ('Feb', 'VEG2'): 130,
    ('Feb', 'OIL1'): 110,
    ('Feb', 'OIL2'): 90,
    ('Feb', 'OIL3'): 115,
    ('Mar', 'VEG1'): 110,
    ('Mar', 'VEG2'): 140,
    ('Mar', 'OIL1'): 130,
    ('Mar', 'OIL2'): 100,
    ('Mar', 'OIL3'): 95,
    ('Apr', 'VEG1'): 120,
    ('Apr', 'VEG2'): 110,
    ('Apr', 'OIL1'): 120,
    ('Apr', 'OIL2'): 120,
    ('Apr', 'OIL3'): 125,
    ('May', 'VEG1'): 100,
    ('May', 'VEG2'): 120,
    ('May', 'OIL1'): 150,
    ('May', 'OIL2'): 110,
    ('May', 'OIL3'): 105,
    ('Jun', 'VEG1'): 90,
    ('Jun', 'VEG2'): 100,
    ('Jun', 'OIL1'): 140,
    ('Jun', 'OIL2'): 80,
    ('Jun', 'OIL3'): 135
}


hardness = {"VEG1": 8.8, "VEG2": 6.1, "OIL1": 2.0, "OIL2": 4.2, "OIL3": 5.0}

price = 150
init_store = 500
target_store = 500
veg_cap = 200
oil_cap = 250

min_hardness = 3
max_hardness = 6
holding_cost = 5

## 模型部署
我们创建一个模型和变量。对于每个时期，我们创建一个变量来考虑生产的食品的价值。对于每个产品（5种油）和每个时期，我们将创建变量来表示购买量、使用量和存储量。

In [3]:
food = gp.Model('Food Manufacture I')
# 每个时期生产的食品数量
produce = food.addVars(months, name="Produce")
# 每个时期购买的各类原料数量
buy = food.addVars(months, oils, name = "Buy")
# 每个时期使用的各类原料数量
consume = food.addVars(months, oils, name = "Use")
# 每个时期储存的各类原料数量
store = food.addVars(months, oils, name = "Store")

Set parameter LicenseID to value 2601452


接下来，我们插入约束条件。平衡约束确保前一期储存的油量加上购买的量等于当前期使用的量加上储存的量（对于每种油）。

In [4]:
#1. 初始平衡约束
Balance0 = food.addConstrs((init_store + buy[months[0], oil]
                 == consume[months[0], oil] + store[months[0], oil]
                 for oil in oils), "Initial_Balance")

#2. 平衡约束
Balance = food.addConstrs((store[months[months.index(month)-1], oil] + buy[month, oil]
                 == consume[month, oil] + store[month, oil]
                 for oil in oils for month in months if month != month[0]), "Balance")

库存目标约束强制要求在最后一期结束时，储存的每种产品数量等于初始数量。

In [5]:
#3. 库存目标
TargetInv = food.addConstrs((store[months[-1], oil] == target_store for oil in oils),"End_Balance")

产能约束限制了每期可以加工的植物油和非植物油的数量。

In [6]:
#4.1 植物油产能约束
VegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "VEG" in oil)
                 <= veg_cap for month in months), "Capacity_Veg")

#4.2 非植物油产能约束
NonVegCapacity = food.addConstrs((gp.quicksum(consume[month, oil] for oil in oils if "OIL" in oil)
                 <= oil_cap for month in months), "Capacity_Oil")

硬度约束限制最终产品的硬度，需要保持在3到6之间。

In [7]:
#5. 硬度
HardnessMin = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 >= min_hardness*produce[month] for month in months), "Hardness_lower")
HardnessMax = food.addConstrs((gp.quicksum(hardness[oil]*consume[month, oil] for oil in oils)
                 <= max_hardness*produce[month] for month in months), "Hardness_upper")

质量守恒约束确保每期使用的油量等于该期生产的食品数量。

In [8]:
#6. 质量守恒
MassConservation = food.addConstrs((consume.sum(month) == produce[month] for month in months), "Mass_conservation")

目标是最大化公司的利润，包括收入以及购买和存储所用产品的成本。

In [9]:
#0. 目标函数
obj = price*produce.sum() - buy.prod(cost) - holding_cost*store.sum()
food.setObjective(obj, GRB.MAXIMIZE) # 最大化利润

接下来，我们开始优化，Gurobi将找到最优解。

In [10]:
food.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 70 rows, 96 columns and 278 nonzeros
Model fingerprint: 0x7d311b4b
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+02]
Presolve removed 33 rows and 45 columns
Presolve time: 0.01s
Presolved: 37 rows, 51 columns, 149 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7375000e+05   1.703125e+03   0.000000e+00      0s
      36    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.078425926e+05


---
## 分析

我们可以期望的最高利润是 $\$107,842.6$。让我们看看实现该数字所需的最优解（存在多个最优解）：

### 采购计划

此计划定义了在规划期内需要购买的植物油（VEG）和非植物油（OIL）的数量。例如，在六月需要购买659.3吨VEG1类型的植物油。

In [11]:
rows = months.copy()
columns = oils.copy()
purchase_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in buy.keys():
    if (abs(buy[month, oil].x) > 1e-6):
        purchase_plan.loc[month, oil] = np.round(buy[month, oil].x, 1)
purchase_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,0.0,0.0,0.0,0.0
Feb,0.0,0.0,0.0,250.0,0.0
Mar,0.0,0.0,0.0,0.0,0.0
Apr,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,0.0,0.0,500.0
Jun,659.3,540.7,0.0,750.0,0.0


### 月度消耗量

此计划确定了规划期内消耗的植物油（VEG）和非植物油（OIL）的数量。例如，一月消耗200吨VEG2类型的植物油。

In [12]:
rows = months.copy()
columns = oils.copy()
reqs = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in consume.keys():
    if (abs(consume[month, oil].x) > 1e-6):
        reqs.loc[month, oil] = np.round(consume[month, oil].x, 1)
reqs

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,0.0,200.0,0.0,0.0,250.0
Feb,85.2,114.8,0.0,0.0,250.0
Mar,96.3,103.7,0.0,250.0,0.0
Apr,159.3,40.7,0.0,250.0,0.0
May,159.3,40.7,0.0,250.0,0.0
Jun,159.3,40.7,0.0,250.0,0.0


### 库存计划

此计划反映了规划期内每个期末植物油（VEG）和非植物油（OIL）的库存量。例如，在二月末我们有500吨OIL1类型的非植物油。

In [13]:
rows = months.copy()
columns = oils.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, oil in store.keys():
    if (abs(store[month, oil].x) > 1e-6):
        store_plan.loc[month, oil] = np.round(store[month, oil].x, 1)
store_plan

,VEG1,VEG2,OIL1,OIL2,OIL3
Jan,500.0,300.0,500.0,500.0,250.0
Feb,414.8,185.2,500.0,750.0,0.0
Mar,318.5,81.5,500.0,500.0,0.0
Apr,159.3,40.7,500.0,250.0,0.0
May,0.0,0.0,500.0,0.0,500.0
Jun,500.0,500.0,500.0,500.0,500.0


**注意：** 如果你想将解决方案写入文件，而不是打印到终端，可以使用model.write()命令。示例实现为：

`food.write("food-manufacture-1-output.sol")`

---
## 参考文献

H. Paul Williams，《数学规划中的模型构建》第五版。

版权所有 © 2020 Gurobi Optimization, LLC